## Installing required libraries

In [ ]:
!pip install --quiet --upgrade langchain langchain-community gradio pypdf langchain-openai faiss-cpu

## loading credentials from .env

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('azure_credentials.env')

EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")
EMBEDDING_ENDPOINT = os.getenv("EMBEDDING_ENDPOINT")
EMBEDDING_API_VERSION = os.getenv("EMBEDDING_API_VERSION")
EMBEDDING_API_KEY = os.getenv("EMBEDDING_API_KEY")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
API_VERSION = os.getenv("API_VERSION")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

## Importing required libraries

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_core.runnables import RunnableParallel

from langchain_core.prompts import ChatPromptTemplate

from langchain_community.document_loaders import PyPDFLoader

from utils import *

## Defining LLMs

In [ ]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

llm_2 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY  ,
    api_version=API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
    )

## Prompts

In [ ]:
from utils import prompt_ as rag_prompt, simple_prompt


## Getting Vector DB's Retriever

In [ ]:

retriever = get_retriever(["book_name.pdf"], embeddings=embeddings)

## Chains

In [ ]:
from utils import get_rag_chain


# Define the second chain with LLM 2
rag_chain_1 = get_rag_chain(retriever, llm, prompt_)
rag_chain_2 = get_rag_chain(retriever, llm2, prompt_)

# Define the third chain with LLM, without RAG
chain3 = (
    simple_prompt
    | llm2  # Second LLM instance
    | StrOutputParser()
)

# Run all chains in parallel with RunnableParallel
parallel_chain = RunnableParallel(
    {"GPT_4o_MINI": rag_chain, "GPT_4o": rag_chain_2, "GP 4o Raw": chain3}
)


# Gradio

In [ ]:
import gradio as gr

# List for storing prev asked questions

In [ ]:
qa_history = []

In [ ]:
# Function to call the chains and storing the answers in list.
def generate_response(message, _):
    global Questions_Answers
    responses = parallel_chain.invoke(message)
    qa_history.append({"question": message, "response": responses})

    return responses["GPT_4o_MINI"], responses["GPT_4o"], responses["GP 4o Raw"]

In [ ]:
# Function to filter previous responses based on a search term
def search_history(search_term):
    # If search_term is empty, return all questions
    if search_term == "":
        return [(qa["question"], qa["response"]) for i, qa in enumerate(qa_history)]

    # Filter questions that contain the search term
    filtered_history = [(qa["question"], str(qa["response"])) for qa in qa_history if search_term.lower() in qa["question"].lower()]
    return filtered_history

### Gradio Interface

In [ ]:
with gr.Blocks() as files:
    file_output = gr.File()
    upload_button = gr.UploadButton("Click to Upload a File", file_types=["image", "video"], file_count="multiple")
    upload_button.upload(upload_file, upload_button, file_output)

# Gradio Interface for Chatbot Tab
with gr.Blocks(fill_width=True, fill_height=True) as chatbot_tab:
    gr.Markdown("### Chatbot")

    # Input box for user question
    question_input = gr.Textbox(label="Ask a question")


    gr.HTML("<hr>")
    gr.Markdown("GPT 4o Mini RAG response")
    response1 = gr.Markdown("")


    gr.HTML("<hr>")
    gr.Markdown("GPT 4o RAG response")
    response2 = gr.Markdown("")

    gr.HTML("<hr>")
    gr.Markdown("GPT 4o response")
    response3 = gr.Markdown("")

    gr.HTML("<hr>")

    # Button to submit question
    submit_button = gr.Button("Submit")

    # Connect question input to chatbot function and display response
    submit_button.click(fn=generate_response,
                        inputs=question_input, outputs=[response1, response2, response3])


# Gradio Interface for Previous Responses Tab
with gr.Blocks() as history_tab:
    gr.Markdown("### Previous Responses")

    # Search bar to filter questions
    search_input = gr.Textbox(label="Search questions")

    # Display previous questions in rows
    questions_output = gr.Dataframe(headers=["Question", "Response"], interactive=False)

    # Update questions output based on search term
    search_input.change(fn=search_history, inputs=search_input, outputs=questions_output)


# Combine the tabs into a single interface
with gr.Blocks() as app:
    with gr.Tabs():
        with gr.TabItem("Chatbot"):
            chatbot_tab.render()

        with gr.TabItem("Previous Responses"):
            history_tab.render()

In [ ]:
app.launch()